In [ ]:
from transformers import AutoTokenizer, AutoModel
import importlib
from transformers import BertModel, BertConfig, PreTrainedTokenizer, BasicTokenizer, BertForTokenClassification, BertForSequenceClassification
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import Dataset
from torch.utils.data import DataLoader

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import numpy as np
from scipy.special import softmax
import pandas as pd
from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
!git clone https://github.com/AIRI-Institute/GENA_LM.git

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('AIRI-Institute/gena-lm-bert-base-t2t')
model = BertForSequenceClassification.from_pretrained('AIRI-Institute/gena-lm-bert-base')
gena_module_name = model.__class__.__module__
cls = getattr(importlib.import_module(gena_module_name), 'BertForSequenceClassification')
model = cls.from_pretrained('AIRI-Institute/gena-lm-bert-base-t2t', num_labels=2, output_attentions=True)

In [ ]:
device = "cuda"

In [ ]:
class HG38Dataset(torch.utils.data.Dataset):

    def __init__(
        self,
        split,
        path,
        max_length=128,
        tokenizer=None,
    ):

        self.max_length = max_length
        self.tokenizer = tokenizer

        df_raw = pd.read_csv(path, sep = '\t')
        self.df = df_raw[df_raw['split'] == split]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        seq, target = (row[0], row[1])
        seq = tokenizer(seq, padding="max_length", truncation=True, max_length=self.max_length)
        seq = seq["input_ids"]
        seq = torch.LongTensor(seq)
        target = torch.LongTensor([target])
        return seq, target

In [ ]:
sns.set_style('whitegrid')
plt.rcParams.update({'font.size': 12})

def plot_losses(train_losses, test_losses, train_accuracies, test_accuracies):
    clear_output()
    fig, axs = plt.subplots(1, 2, figsize=(13, 4))
    axs[0].plot(np.arange(1, len(train_losses) + 1) * 100, train_losses, label='train')
    axs[0].plot(np.arange(1, len(test_losses) + 1) * 100, test_losses, label='val')
    axs[0].set_ylabel('loss')

    axs[1].plot(np.arange(1, len(train_accuracies) + 1) * 100, train_accuracies, label='train')
    axs[1].plot(np.arange(1, len(test_accuracies) + 1) * 100, test_accuracies, label='val')
    axs[1].set_ylabel('accuracy')

    for ax in axs:
        ax.set_xlabel('Number of training steps performed')
        ax.legend()

    fig.suptitle('GENA-LM')

    plt.show()

In [ ]:
num_epochs = 10
max_length = 128
path = 'https://drive.google.com/file/d/1hYaaF4DSMaWMogSZJVlMKH85bL15Thb-/view?usp=drive_link'
path = 'https://drive.google.com/uc?id=' + path.split('/')[-2]
batch_size = 32
learning_rate = 5e-06
weight_decay = 0.0

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

ds_train = HG38Dataset(
    split = 'train',
    path = path,
    max_length = max_length,
    tokenizer=tokenizer
)

ds_test = HG38Dataset(
    split = 'val',
    path = path,
    max_length = max_length,
    tokenizer=tokenizer
)

train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=False, drop_last=True)

loss_fn = nn.CrossEntropyLoss()

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

model.to(device)

train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []

train_loss, train_accuracy = 0.0, 0.0

for epoch in range(num_epochs):

    model.train()
    for batch_ndx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        if (batch_ndx + 1) == 1201:
          break

        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output['logits'], target.squeeze())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        pred = output['logits'].argmax(dim=1, keepdim=True)
        train_accuracy += pred.eq(target.view_as(pred)).sum().item()

        if (batch_ndx + 1) % 100 == 0:

            train_loss /= batch_size * 100
            train_accuracy /= batch_size * 100
            train_losses += [train_loss]
            train_accuracies += [train_accuracy]

            test_loss, test_accuracy = 0.0, 0.0
            model.eval()
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)

                with torch.no_grad():
                    output = model(data)
                    loss = loss_fn(output['logits'], target.squeeze())

                test_loss += loss.item()
                pred = output['logits'].argmax(dim=1, keepdim=True)
                test_accuracy += pred.eq(target.view_as(pred)).sum().item()

            test_loss /= len(test_loader.dataset)
            test_accuracy /= len(test_loader.dataset)
            test_losses += [test_loss]
            test_accuracies += [test_accuracy]
            plot_losses(train_losses, test_losses, train_accuracies, test_accuracies)

            train_loss, train_accuracy = 0.0, 0.0

In [ ]:
ds_test = HG38Dataset(
    split = 'test',
    path = path,
    max_length = max_length,
    tokenizer=tokenizer
)

test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=False, drop_last=True)

outputs = []
targets = []

model.eval()
for data, target in test_loader:
    data, target = data.to(device), target.to(device)

    with torch.no_grad():
        output = model(data)
        outputs += output['logits'].cpu().tolist()
        targets += target.cpu().tolist()

In [ ]:
from bertviz import head_view, model_view

# tokenizer = DNATokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('AIRI-Institute/gena-lm-bert-base-t2t')
sentence = "TCGGAACCGTGCTTGCAGCCCCCGGCTGGC"
inputs = tokenizer.encode(sentence, return_tensors='pt', add_special_tokens=False)
model = model.to(device)
outputs = model(inputs.to(device))
attention = outputs[-1]
tokens = tokenizer.convert_ids_to_tokens(inputs[0])

In [ ]:
head_view(attention, tokens)